## Importing Libraries


In [1]:
!pip -q install simpletransformers

In [2]:
!pip -q install bs4

In [3]:
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import sklearn
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

## Loading Files From Dataset

In [4]:
df = pd.read_csv("../input/abusive-comment-detection/Abusive_Comment_Tamil_train.csv",header=None,sep='\t')
df_eval = pd.read_csv("../input/abusive-comment-detection/Abusive_Comment_Tamil_dev.csv",header=None,sep='\t')
df_test = pd.read_csv("../input/abusive-comment-detection/Abusive_Comment_Tamil_test.csv",header=None,sep='\t')

In [5]:
df_eval.rename(columns={0:'Labels',1:'Text'},inplace=True)
df_eval = df_eval[['Text','Labels']]
df.rename(columns={0:'Labels',1:'Text'},inplace=True)
df = df[['Text','Labels']]
df_test.rename(columns={0:'Labels',1:'Text'},inplace=True)
df_test = df_test[['Text','Labels']]

In [6]:
num_labels = len(df['Labels'].unique())
keys = list(df['Labels'].unique())
values = list(range(0, num_labels))
label_dict = dict(zip(keys,values))
df['Labels'] = df['Labels'].apply(lambda x:label_dict[x])
df_eval['Labels'] = df_eval['Labels'].apply(lambda x:label_dict[x])
df_test['Labels'] = df_test['Labels'].apply(lambda x:label_dict[x])

## Balancing the imbalanced dataset

In [7]:
def oversample(df):
        classes = df['Labels'].value_counts().to_dict()
        most = max(classes.values())
        classes_list = []
        for key in classes:
            classes_list.append(df[df['Labels'] == key]) 
        classes_sample = []
        for i in range(1,len(classes_list)):
            classes_sample.append(classes_list[i].sample(most, replace=True))
        df_maybe = pd.concat(classes_sample)
        final_df = pd.concat([df_maybe,classes_list[0]], axis=0)
        final_df = final_df.reset_index(drop=True)
        return pd.DataFrame({'Text': final_df['Text'].tolist(), 'Labels': final_df['Labels'].tolist()})

In [8]:
def over_under_sample(df):
  unq_labels = list(set(df['Labels'].tolist()))
  texts = df['Text'].tolist()
  labels = df['Labels'].tolist()
  data_dict = dict()

  for l in unq_labels:
    data_dict[l] = []

  for i in range(len(texts)):
    data_dict[labels[i]].append(texts[i])

  req_len = len(labels)//len(unq_labels)

  for label in data_dict:
    if len(data_dict[label]) > req_len:
      data_dict[label] = data_dict[label][:req_len]

    new_texts = []

    new_labels = []      
    for l in data_dict:
      new_texts += data_dict[l]
      new_labels += [l]*len(data_dict[l])
    return oversample(pd.DataFrame({'Text': new_texts, 'Labels': new_labels}))

In [9]:
df = over_under_sample(df)

## Model Training

In [10]:
model_args = ClassificationArgs()

In [11]:
model_args.overwrite_output_dir=True
model_args.eval_batch_size=8
model_args.train_batch_size=8
model_args.learning_rate=4e-5



In [12]:
model = ClassificationModel(
    'bert',
    'google/muril-base-cased',
    num_labels=9,
    args=model_args,
    tokenizer_type="bert",
    tokenizer_name='google/muril-base-cased'
) 

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/909M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not in

Downloading:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [13]:
for i in range(0,4):
    !rm -rf /kaggle/working/outputs
    model.train_model(df,eval_data=df_eval,acc=sklearn.metrics.classification_report)
    result, model_outputs, preds_list = model.eval_model(df_test,acc=sklearn.metrics.classification_report)
    for j in result.values():
        print(j)

/opt/conda/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/11664 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1458 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:922: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm
/opt/conda/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1458 [00:00<?, ?it/s]

  0%|          | 0/700 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/88 [00:00<?, ?it/s]

0.4653093358796267
              precision    recall  f1-score   support

         0.0       0.36      0.31      0.33        26
         1.0       0.78      0.85      0.81       417
         2.0       0.67      0.50      0.57         8
         3.0       0.58      0.62      0.60       127
         4.0       0.42      0.38      0.40        47
         5.0       0.59      0.33      0.43        48
         6.0       0.55      0.24      0.33        25
         7.0       0.00      0.00      0.00         2

    accuracy                           0.69       700
   macro avg       0.49      0.40      0.43       700
weighted avg       0.68      0.69      0.68       700

2.247648542069576


## Predicting Results


In [14]:
df_test = df_test.dropna()
df_test = df_test.reset_index()
df_test = df_test.drop(['index'],axis=1)

In [15]:
predictions, raw_outputs = model.predict(df_test['Text'].to_list())

  0%|          | 0/699 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

In [16]:
df_final = df_test.copy()
reverse_label_dict = {v:u for u,v in label_dict.items()}
df_final['Predicted_Labels'] = predictions
df_final['Predicted_Labels'] = df_final['Predicted_Labels'].apply(lambda x:reverse_label_dict[x])
df_final['Labels'] = df_final['Labels'].apply(lambda x:reverse_label_dict[x])
df_final['pid'] = df_final.index
df_final = df_final[['pid','Predicted_Labels','Labels']]

In [17]:
df_final

,pid,Predicted_Labels,Labels
0,0,None-of-the-above,Xenophobia
1,1,Xenophobia,Xenophobia
2,2,Misandry,Misandry
3,3,Counter-speech,Counter-speech
4,4,None-of-the-above,None-of-the-above
...,...,...,...
694,694,None-of-the-above,None-of-the-above
695,695,None-of-the-above,None-of-the-above
696,696,Counter-speech,None-of-the-above
697,697,Misandry,Misogyny


In [18]:
score = f1_score(df_final['Labels'],df_final['Predicted_Labels'],average='weighted')
print("The weighted average is:" + str(score))

The weighted average is:0.6805235158121308
